In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class SendEmailInput(BaseModel):
    to: str = Field(description="Comma seperared list of users emails to which email needs to be sent")
    email: str = Field(description="The email text to be sent")

class SendEmailTool(BaseTool):
    name = "email_triage"
    description = "useful for when you need to send an email to someone"
    args_schema: Type[BaseModel] = SendEmailInput

    def _run(
        self, 
        email: str,
        to: str
    ) -> str:
        print(email)
        return f"Email sent successfully to {to}"

    async def _arun(
        self, 
        email: str,
        to: str
    ) -> str:
        print(email)
        return f"Email sent successfully to {to}"

In [11]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
session = FloSession(llm)

session.register_tool(
    name="SendEmailTool", 
    tool=SendEmailTool()
)

In [17]:
agent_yaml = """
apiVersion: flo/alpha-v1
kind: FloRoutedTeam
name: invite-handler
team:
    name: EventInviteBot
    router:
        name: Sequential-Router
        kind: linear
    agents:
      - name: EmailFriends
        job: You job is to send an invite to the christmas party at my house to my friends and friends only, not collegues, invite their spouses too. Keep the email warm and friendly.
        role: personal ai assistant
        tools:
          - name: SendEmailTool
      - name: EmailColleagues
        job: You job is to send an invite to the christmas party at my house to my colleagues and not friends. Keep the email formal, and DO NOT invite the spouses.
        role: office ai assistant
        tools:
          - name: SendEmailTool
"""

In [18]:
input_prompt = """
Here is the list of user emails and there relations to me

vishnu@gmail.com / friend
nk@gmail.com / friend
jk@gmail.com / colleague
ck@hotmail.com / friend
hk@gmail.com / colleague
jak@gmail.com / colleague
ck@gmail.com / friend.

Please invite these nice folks to my christmas party
"""

In [19]:
flo: Flo = Flo.build(session, yaml=agent_yaml)
for s in flo.stream(input_prompt):
     if "__end__" not in s:
        print(s)
        print("----")



> Entering new AgentExecutor chain...

Invoking: `email_triage` with `{'to': 'vishnu@gmail.com', 'email': "Hey Vishnu,\n\nI hope this email finds you well! I'm excited to invite you and your spouse to my Christmas party at my house. It's going to be a fun and festive evening, and I'd love for you both to join us.\n\nLooking forward to celebrating together!\n\nBest,\n[Your Name]"}`


Hey Vishnu,

I hope this email finds you well! I'm excited to invite you and your spouse to my Christmas party at my house. It's going to be a fun and festive evening, and I'd love for you both to join us.

Looking forward to celebrating together!

Best,
[Your Name]
Email sent successfully to vishnu@gmail.com
Invoking: `email_triage` with `{'to': 'nk@gmail.com', 'email': "Hey NK,\n\nI hope this email finds you well! I'm excited to invite you and your spouse to my Christmas party at my house. It's going to be a fun and festive evening, and I'd love for you both to join us.\n\nLooking forward to celebrating